In [ ]:
from fastapi import FastAPI
from starlette.responses import StreamingResponse
import cv2
import time
import uvicorn
import nest_asyncio
from win32api import GetSystemMetrics
import numpy as np
from PIL import ImageGrab
import datetime

nest_asyncio.apply()
app = FastAPI()

def generate_frames():
    width = GetSystemMetrics(0)
    height = GetSystemMetrics(1)
    fps = 10  # Assuming 10 frames per second
    
    frame_count = 0  # Initialize frame_count
    extracted_F= 2
    real = 0
    last_emit_time = time.time()  # Initialize the last emission time
    while True:
        img = ImageGrab.grab()  # Capture the screen
        img = np.array(img)
        img_final = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Increment frame count
        frame_count += 1
        res = frame_count % fps
        d = fps/extracted_F
        current_time = time.time()
        # elapsed_time = current_time - last_emit_time
        # if elapsed_time >= 1 / extracted_F:
        if res%d==0.0:
                real+=1
                print(real)
                # Resize the frame to 480p resolution
                # img_final = cv2.resize(img_final, (854, 480))  # 480p resolution is 854x480
                img_final = cv2.resize(img_final, (640, 360))  # 480p resolution is 854x480
                # Convert the frame to JPEG format for streaming
                _, buffer = cv2.imencode('.jpg', img_final)
                
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')
            
@app.get("/video_feed")
async def video_feed():
    return StreamingResponse(generate_frames(), media_type="multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)
